# Step 0: Import Libraries 

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers 
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model

import os as os
import numpy as np
import matplotlib.pyplot as plt

# Dataset sizes
num_train_images = 23000
num_validation_images = 2000
num_test_images = 399

# Some Hyperparameters
input_image_size = 256
batch_size = 32
num_epochs = 1
learning_rate = 0.001
l2_lambda = 0.01

# Parameters derived from hyperparameters
training_steps_per_epoch = int(num_train_images/batch_size)
validation_steps = int(num_validation_images/batch_size)
testing_steps_per_epoch = num_test_images/batch_size

%matplotlib inline

 # Step 1: Load the Inception V3 network, and modify it for transfer learning.


In [3]:
# Instantiate an InceptionV3() object
base_model = InceptionV3(weights='imagenet', input_shape=(input_image_size, input_image_size, 3), include_top=False)

x = base_model.output
# Add a GlobalAveragePooling2D() object to the network

x = GlobalAveragePooling2D()(x)

x = Flatten()(x)

# Add a Dense() layer.
x = Dense(1024, activation='relu')(x)

# Add another dense layer with 1 unit and the sigmoid activation function
predictions = Dense(1, activation='sigmoid')(x)

# Wrap our network layers in a model object
model = Model(inputs=base_model.input, outputs=predictions)

# Set all the layers in the base model to be non-trainable.  
for layer in base_model.layers:
    layer.trainable = False;
        
# Create an Adam optimizer object with learning rate equal to learning_rate 
optimizer = Adam(lr=learning_rate)


# Compile the CNN using model.compile() 
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

print('Model compiled!')

Model compiled!


# Step 2: Load Images and Prepare the Network for training

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=True
)


# Create the validation dataset generator
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load the training set
training_set = train_datagen.flow_from_directory(
    "C:\\Users\\ducth\\cmpt 487 asm\\asm6\\train" ,
    target_size=(input_image_size, input_image_size),
    batch_size=batch_size,
    class_mode='binary'
)

# Load the validation dataset 
validation_set = validation_datagen.flow_from_directory(
    "C:\\Users\\ducth\\cmpt 487 asm\\asm6\\valid" ,
    target_size=(input_image_size, input_image_size),
    batch_size=batch_size,
    class_mode='binary'
)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


# Step 3: Run the CNN Network

In [8]:
# Train the CNN 
history = model.fit(
    training_set,
    steps_per_epoch=training_steps_per_epoch,
    epochs=num_epochs,
    validation_data=validation_set,
    validation_steps=validation_steps,
    verbose=1
)


718/718 [==============================] - 781s 1s/step - loss: 0.0553 - accuracy: 0.9827 - val_loss: 0.0342 - val_accuracy: 0.9874


# Step 4: Save the model and weights for prediction

In [10]:
model.save('Cat-Dog-transfer-Inception.h5')
print("Saved model to disk")

C:\Users\ducth\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saved model to disk


 # Step 5: Predict Dog/Cat using the Trained Model

In [16]:
model = load_model('Cat-Dog-transfer-Inception.h5')
test_datagen = ImageDataGenerator(rescale=1./255)

generator = test_datagen.flow_from_directory(
    "C:\\Users\\ducth\\cmpt 487 asm\\asm6\\test1",
    target_size=(input_image_size, input_image_size),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False  
)

probabilities = model.predict_generator(generator)

y_pred = np.squeeze((probabilities > 0.5).astype(int))

accuracy = np.sum(y_pred == generator.classes) / len(generator.classes)
accuracy_percentage = (accuracy * 100)

print(f'The classification rate is {accuracy_percentage:.2f}%')

Found 399 images belonging to 2 classes.


C:\Users\ducth\AppData\Local\Temp\ipykernel_18108\4285891948.py:16: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  probabilities = model.predict_generator(generator)


The classification rate is 98.25%
